In [5]:
import pickle
from pathlib import Path
import numpy as np
import torch
%load_ext autoreload
%autoreload 2
from tools.dataset_util import Dataset
dataset = Dataset("train", r"/mnt/e/DataSet/kitti")
root_dir = Path(r"/mnt/e/DataSet/kitti/kitti_inst_database")

with open(root_dir / "kitti_car_database.pkl", "rb") as f:
    db = pickle.load(f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from tools.box_util import rect2lidar, get_objects_in_boxes3d
save_dir = root_dir / "pc_origin"

for sample in db.values():
    name = sample['name']
    print(name)
    idx = int(name.split('_')[0])
    calib = sample['calib']
    lidar = dataset.get_lidar(idx)[:, :3]
    label = sample['label']
    bbox3d = np.array([[*label.pos, label.l, label.h, label.w, label.ry]])
    bbox3d_in_lidar = rect2lidar(bbox3d, calib)
    
    object = get_objects_in_boxes3d(lidar, bbox3d_in_lidar, enlarge=torch.tensor([0.4, 0.2, 0]))[0]
    
    file_name = f"{name}.bin"
    object.tofile(str(save_dir / file_name))
    

000003_0
000006_2
000007_0
000008_5
000009_0
000010_1
000010_2
000010_4
000011_0
000013_0
000019_1
000020_0
000021_0
000021_3
000025_1
000025_3
000031_0
000031_1
000031_4
000032_0
000032_1
000034_0
000035_1
000036_0
000037_0
000038_0
000038_5
000039_1
000044_1
000045_0
000045_1
000046_0
000047_0
000047_2
000047_4
000049_0
000049_1
000050_0
000050_1
000052_0
000052_5
000053_1
000053_2
000053_3
000053_5
000059_2
000060_1
000062_0
000063_0
000064_0
000064_1
000067_3
000067_4
000069_0
000071_0
000072_1
000075_1
000079_3
000080_0
000080_3
000083_1
000083_2
000083_3
000084_0
000087_0
000089_0
000090_0
000092_1
000093_1
000094_0
000095_0
000095_1
000098_0
000098_1
000099_2
000099_3
000101_0
000101_2
000104_0
000107_3
000108_0
000110_3
000110_4
000111_0
000111_3
000113_4
000113_5
000113_7
000114_0
000116_0
000116_1
000116_2
000117_0
000119_0
000119_2
000120_2
000120_6
000122_0
000126_0
000127_0
000128_1
000129_0
000129_1
000131_1
000134_0
000138_0
000138_1
000140_0
000140_1
000141_0
000144_1
0

In [27]:
sample = np.fromfile(str(save_dir / "007479_2.bin"), dtype=np.float32).reshape(-1, 3)
sample.shape

(1587, 3)

In [28]:

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(sample)
o3d.visualization.draw_geometries([pcd])

In [31]:

def to_corners(bbox3d):
    l, w, h = bbox3d[3:6]
    xs = [-l/2, -l/2, l/2, l/2, -l/2, -l/2, l/2, l/2]
    ys = [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2]
    zs = [-h/2, -h/2, -h/2, -h/2, h/2, h/2, h/2, h/2]
    
    ry = bbox3d[6]
    R = np.array([[np.cos(ry), -np.sin(ry), 0], [np.sin(ry), np.cos(ry), 0], [0, 0, 1]])
    
    xyz = bbox3d[:3]
    
    corners = np.vstack([xs, ys, zs])
    corners = np.dot(R, corners).T + xyz
    
    return corners
    

In [32]:

for sample in db.values():
    name = sample['name']
    print(name)
    idx = int(name.split('_')[0])
    calib = sample['calib']
    label = sample['label']
    bbox3d = np.array([[*label.pos, label.l, label.h, label.w, label.ry]])
    bbox3d_in_lidar = rect2lidar(bbox3d, calib).reshape(-1)
    
    corners = to_corners(bbox3d_in_lidar)
    np.savetxt(str(root_dir / 'cornners' / f"{name}.txt"), corners)
    



000003_0
000006_2
000007_0
000008_5
000009_0
000010_1
000010_2
000010_4
000011_0
000013_0
000019_1
000020_0
000021_0
000021_3
000025_1
000025_3
000031_0
000031_1
000031_4
000032_0
000032_1
000034_0
000035_1
000036_0
000037_0
000038_0
000038_5
000039_1
000044_1
000045_0
000045_1
000046_0
000047_0
000047_2
000047_4
000049_0
000049_1
000050_0
000050_1
000052_0
000052_5
000053_1
000053_2
000053_3
000053_5
000059_2
000060_1
000062_0
000063_0
000064_0
000064_1
000067_3
000067_4
000069_0
000071_0
000072_1
000075_1
000079_3
000080_0
000080_3
000083_1
000083_2
000083_3
000084_0
000087_0
000089_0
000090_0
000092_1
000093_1
000094_0
000095_0
000095_1
000098_0
000098_1
000099_2
000099_3
000101_0
000101_2
000104_0
000107_3
000108_0
000110_3
000110_4
000111_0
000111_3
000113_4
000113_5
000113_7
000114_0
000116_0
000116_1
000116_2
000117_0
000119_0
000119_2
000120_2
000120_6
000122_0
000126_0
000127_0
000128_1
000129_0
000129_1
000131_1
000134_0
000138_0
000138_1
000140_0
000140_1
000141_0
000144_1
0

In [29]:
name = '007474_0'
name = '007479_2'
idx = int(name.split('_')[0])
bbox = np.loadtxt(str(root_dir / 'cornners' / f"{name}.txt"))
label = db[name]['label']
calib = dataset.get_calib(idx)
from tools.box_util import rect2lidar, get_objects_in_boxes3d
lidar = dataset.get_lidar(idx)[:, :3]
bbox3d = np.array([[*label.pos, label.l, label.h, label.w, label.ry]])
bbox3d_in_lidar = rect2lidar(bbox3d, calib)
corners = to_corners(bbox3d_in_lidar.reshape(-1))

cord = np.concatenate([corners, lidar], axis=0)
from tools.box_util import remove_points_in_boxes3d, check_points_in_boxes3d
no_obj = remove_points_in_boxes3d(lidar, bbox3d_in_lidar)
import torch
obj = get_objects_in_boxes3d(lidar, bbox3d_in_lidar, enlarge=torch.tensor([0.4, 0.2, 0]))[0]

rgb1 = np.zeros_like(obj) + [0, 255, 0]
rgb2 = np.zeros_like(no_obj) + [255, 0, 0]

cord = np.concatenate([obj, no_obj], axis=0)
rgb = np.concatenate([rgb1, rgb2], axis=0) 


In [34]:
from tools.visualize_util import show_o3d
show_o3d(cord, rgb)

In [35]:
for key in db.keys():
    sample = np.fromfile(str(save_dir / f"{key}.bin"), dtype=np.float32).reshape(-1, 3)
    bbox = np.loadtxt(str(root_dir / "cornners" / f"{key}.txt"))
    cord = np.concatenate([sample, bbox], axis=0) 
    show_o3d(cord, rgb)

KeyboardInterrupt: 

In [4]:
pcd_dir = Path(r"/home/a/PoinTr/experiments/PoinTr/KITTI_models/test_example/vis_result")
import os 
from tools.visualize_util import show_o3d
file_list = list(filter(lambda x:not x.endswith(".png"), os.listdir(pcd_dir)))

file_name = file_list[13]
input = np.load(str(pcd_dir / file_name / "input.npy")) 
pred = np.load(str(pcd_dir / file_name / "pred.npy"))

show_o3d(pred)